In [1]:
import datetime
import gc
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import time
import warnings

from contextlib import contextmanager
from pandas.core.common import SettingWithCopyWarning
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold

warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)


FEATS_EXCLUDED = [
    'first_active_month', 'target', 'card_id', 'outliers',
    'hist_purchase_date_max', 'hist_purchase_date_min', 'hist_card_id_size',
    'new_purchase_date_max', 'new_purchase_date_min', 'new_card_id_size',
    'OOF_PRED', 'month_0',
    'new_month_lag_min','quarter','new_month_lag_max','hist_month_lag_max','new_month_diff_var'#40
    'hist_hour_nunique','new_installments_sum','hist_installments_max','new_subsector_id_nunique',
    'new_installments_max','hist_month_lag_min','new_weekofyear_nunique','new_weekend_mean','new_hour_nunique'#30
    'hist_subsector_id_nunique']
@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

# rmse
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns
    
# Display/plot feature importance
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]

    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig('lgbm_importances.png')

# reduce memory
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df
    


# LightGBM GBDT with KFold or Stratified KFold
def kfold_lightgbm(train_df, test_df, num_folds, stratified = False, debug= False):
    print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))

    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=326)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=326)

    # Create arrays and dataframes to store results
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    feature_importance_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in FEATS_EXCLUDED]

    # k-fold
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['outliers'])):
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['target'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['target'].iloc[valid_idx]

        # set data structure
        lgb_train = lgb.Dataset(train_x,
                                label=train_y,
                                free_raw_data=False)
        lgb_test = lgb.Dataset(valid_x,
                               label=valid_y,
                               free_raw_data=False)

        # params optimized by optuna
        params ={
                'task': 'train',
                'boosting': 'goss',
                'objective': 'regression',
                'metric': 'rmse',
                'learning_rate': 0.01,
                'subsample': 0.9855232997390695,
                'max_depth': 7,
                'top_rate': 0.9064148448434349,
                'num_leaves': 63,
                'min_child_weight': 41.9612869171337,
                'other_rate': 0.0721768246018207,
                'reg_alpha': 9.677537745007898,
                'colsample_bytree': 0.5665320670155495,
                'min_split_gain': 9.820197773625843,
                'reg_lambda': 8.2532317400459,
                'min_data_in_leaf': 21,
                'verbose': -1,
                'seed':int(2**n_fold),
                'bagging_seed':int(2**n_fold),
                'drop_seed':int(2**n_fold)
                }

        reg = lgb.train(
                        params,
                        lgb_train,
                        valid_sets=[lgb_train, lgb_test],
                        valid_names=['train', 'test'],
                        num_boost_round=10000,
                        early_stopping_rounds= 200,
                        verbose_eval=100
                        )

        oof_preds[valid_idx] = reg.predict(valid_x, num_iteration=reg.best_iteration)
        sub_preds += reg.predict(test_df[feats], num_iteration=reg.best_iteration) / folds.n_splits

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = np.log1p(reg.feature_importance(importance_type='gain', iteration=reg.best_iteration))
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        print('Fold %2d RMSE : %.6f' % (n_fold + 1, rmse(valid_y, oof_preds[valid_idx])))
        del reg, train_x, train_y, valid_x, valid_y
        gc.collect()

    # display importances
    display_importances(feature_importance_df)

    if not debug:
        # save submission file
        test_df.loc[:,'target'] = sub_preds
        test_df = test_df.reset_index()
        test_df[['card_id', 'target']].to_csv(submission_file_name, index=False)


In [4]:
import lightgbm as lgb
import numpy as np
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split

import optuna

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def objective(trial):
    
    
        num_folds = 11
        train_x, train_y = train_df[feats], train_df['target']
        #data, target = sklearn.datasets.load_breast_cancer(return_X_y=True)
        '''dtrain = lgb.Dataset(train_x, label=train_y)'''
       
        lgbm_train = lgb.Dataset(train_x,
                                 train_y,
                                  free_raw_data=False
                                  )

        params = {'objective': 'regression',
                  'metric': 'rmse',
                  'verbosity': -1,
                  "learning_rate": trial.suggest_uniform('learning_rate', 0.001, 1),
                  
                  'device': 'gpu',
                  'gpu_platform_id': 1,
                  'gpu_device_id': 0,
                  'num_thread' : 1,
                  'sparse_threshold' : 1,
                  
                  'seed': 2779,
                  #'boosting_type': trial.suggest_categorical('boosting', ['gbdt',  'goss']),
                  'num_leaves': trial.suggest_int('num_leaves', 16, 200),
                  #'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.001, 1),
                  'subsample': trial.suggest_uniform('subsample', 0.001, 1),
                  'max_depth': trial.suggest_int('max_depth', 5, 20),
                  'reg_alpha': trial.suggest_uniform('reg_alpha', 0, 10),
                  # 'reg_lambda': trial.suggest_uniform('reg_lambda', 0, 10),
                  #'min_split_gain': trial.suggest_uniform('min_split_gain', 0, 10),
                  #'min_child_weight': trial.suggest_uniform('min_child_weight', 0, 45),
                  #'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 16, 64)
                  
                  'min_child_samples' : trial.suggest_int('min_child_samples', 1, 200),
                  #'num_iterations': trial.suggest_uniform('num_iterations', 1, 5000),
                  'feature_fraction' : trial.suggest_uniform('feature_fraction', 0.001, 1),
                  #'random_state': trial.suggest_int('random_state', 1, 5000),
                  #'max_bin' :  trial.suggest_int('random_state', 1, 256)
                  }

        '''if params['boosting_type'] == 'dart':
            params['drop_rate'] = trial.suggest_loguniform('drop_rate', 1e-8, 1.0)
            params['skip_drop'] = trial.suggest_loguniform('skip_drop', 1e-8, 1.0)
        if params['boosting_type'] == 'goss':
            params['top_rate'] = trial.suggest_uniform('top_rate', 0.0, 1.0)
            params['other_rate'] = trial.suggest_uniform('other_rate', 0.0, 1.0 - params['top_rate'])'''

        #folds = StratifiedKFold(n_splits=NUM_FOLDS, shuffle=True, random_state=47)
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=47)


        clf = lgb.cv(
                        params,
                        lgbm_train,
                        metrics=['rmse'],
                        nfold=num_folds,
                        folds=folds.split(train_df[feats], train_df['outliers']),
                        num_boost_round=10000,
                        early_stopping_rounds= 200,
                        verbose_eval=100,
                        seed=47
            
                         )
        gc.collect()
        return clf['rmse-mean'][-1]

        
        '''gbm = lgb.train(params, dtrain)
        preds = gbm.predict(train_x)
        pred_labels = np.rint(preds)
        error =  rmse(train_y,preds)
        #accuracy = sklearn.metrics.accuracy_score(test_y, pred_labels)
        return error'''

In [5]:
with timer("split train & test"):
        train_df = reduce_mem_usage(pd.read_csv('low_features3.csv'))
        test_df = reduce_mem_usage(pd.read_csv('test_clean3.csv'))
       
        feature_importance_df = pd.DataFrame()
        feats = [f for f in train_df.columns if f not in FEATS_EXCLUDED]
        
        study = optuna.create_study()
        study.optimize(objective, n_trials=100)

        print('Number of finished trials: {}'.format(len(study.trials)))

        print('Best trial:')
        trial = study.best_trial

        print('  Value: {}'.format(trial.value))

        print('  Params: ')
        for key, value in trial.params.items():
            print('    {}: {}'.format(key, value))
        
        hist_df = study.trials_dataframe()
        hist_df.to_csv("optuna_result_lgbm.csv")

        del df
        gc.collect()

Memory usage after optimization is: 26.38 MB
Decreased by 74.1%
Memory usage after optimization is: 23.70 MB
Decreased by 74.7%
[100]	cv_agg's rmse: 3.82791 + 0.0232763
[200]	cv_agg's rmse: 3.8826 + 0.0248887


[I 2019-02-25 11:26:10,993] Finished a trial resulted in value: 3.7134986150705673. Current best value is 3.7134986150705673 with parameters: {'learning_rate': 0.37018241335392194, 'num_leaves': 102, 'subsample': 0.8615201645052978, 'max_depth': 14, 'reg_alpha': 8.339943652931943, 'min_child_samples': 22, 'feature_fraction': 0.37343060042557186}.


[100]	cv_agg's rmse: 4.24597 + 0.0352717
[200]	cv_agg's rmse: 4.41099 + 0.0319728


[I 2019-02-25 11:29:47,894] Finished a trial resulted in value: 3.7082614899740545. Current best value is 3.7082614899740545 with parameters: {'learning_rate': 0.8366374994969353, 'num_leaves': 161, 'subsample': 0.0120844015099608, 'max_depth': 18, 'reg_alpha': 3.491205675249871, 'min_child_samples': 169, 'feature_fraction': 0.3849077264659108}.


[100]	cv_agg's rmse: 3.748 + 0.0357499
[200]	cv_agg's rmse: 3.79479 + 0.03663


...

lightgbm.basic.LightGBMError: Out of Resources
[W 2019-02-25 13:24:45,437] Setting trial status as TrialState.FAIL because of the following error: LightGBMError('Out of Resources',)
Traceback (most recent call last):
  File "C:\Anaconda3\lib\site-packages\optuna\study.py", line 409, in _run_trial
    result = func(trial)
  File "<ipython-input-4-63625ec2837d>", line 75, in objective
    seed=47
  File "C:\Anaconda3\lib\site-packages\lightgbm\engine.py", line 458, in cv
    stratified=stratified, shuffle=shuffle)
  File "C:\Anaconda3\lib\site-packages\lightgbm\engine.py", line 317, in _make_n_folds
    cvbooster = Booster(tparam, train_set)
  File "C:\Anaconda3\lib\site-packages\lightgbm\basic.py", line 1541, in __init__
    ctypes.byref(self.handle)))
  File "C:\Anaconda3\lib\site-packages\lightgbm\basic.py", line 46, in _safe_call
    raise LightGBMError(decode_string(_LIB.LGBM_GetLastError()))
lightgbm.basic.LightGBMError: Out of Resources
[W 2019-02-25 13:24:46,987] Setting trial st

lightgbm.basic.LightGBMError: Out of Resources
[W 2019-02-25 13:24:59,392] Setting trial status as TrialState.FAIL because of the following error: LightGBMError('Out of Resources',)
Traceback (most recent call last):
  File "C:\Anaconda3\lib\site-packages\optuna\study.py", line 409, in _run_trial
    result = func(trial)
  File "<ipython-input-4-63625ec2837d>", line 75, in objective
    seed=47
  File "C:\Anaconda3\lib\site-packages\lightgbm\engine.py", line 458, in cv
    stratified=stratified, shuffle=shuffle)
  File "C:\Anaconda3\lib\site-packages\lightgbm\engine.py", line 317, in _make_n_folds
    cvbooster = Booster(tparam, train_set)
  File "C:\Anaconda3\lib\site-packages\lightgbm\basic.py", line 1541, in __init__
    ctypes.byref(self.handle)))
  File "C:\Anaconda3\lib\site-packages\lightgbm\basic.py", line 46, in _safe_call
    raise LightGBMError(decode_string(_LIB.LGBM_GetLastError()))
lightgbm.basic.LightGBMError: Out of Resources
[W 2019-02-25 13:25:00,943] Setting trial st

lightgbm.basic.LightGBMError: Out of Resources
[W 2019-02-25 13:25:13,322] Setting trial status as TrialState.FAIL because of the following error: LightGBMError('Out of Resources',)
Traceback (most recent call last):
  File "C:\Anaconda3\lib\site-packages\optuna\study.py", line 409, in _run_trial
    result = func(trial)
  File "<ipython-input-4-63625ec2837d>", line 75, in objective
    seed=47
  File "C:\Anaconda3\lib\site-packages\lightgbm\engine.py", line 458, in cv
    stratified=stratified, shuffle=shuffle)
  File "C:\Anaconda3\lib\site-packages\lightgbm\engine.py", line 317, in _make_n_folds
    cvbooster = Booster(tparam, train_set)
  File "C:\Anaconda3\lib\site-packages\lightgbm\basic.py", line 1541, in __init__
    ctypes.byref(self.handle)))
  File "C:\Anaconda3\lib\site-packages\lightgbm\basic.py", line 46, in _safe_call
    raise LightGBMError(decode_string(_LIB.LGBM_GetLastError()))
lightgbm.basic.LightGBMError: Out of Resources
[W 2019-02-25 13:25:14,881] Setting trial st

Number of finished trials: 100
Best trial:
  Value: 3.6539518322613542
  Params: 
    learning_rate: 0.11610799508878916
    num_leaves: 77
    subsample: 0.46259974714863705
    max_depth: 8
    reg_alpha: 7.6055284337946985
    min_child_samples: 85
    feature_fraction: 0.9238425852119821


NameError: name 'df' is not defined